In [ ]:
# 10 selected MoAs 
moas_to_use = ['Aurora kinase inhibitor', 'tubulin polymerization inhibitor', 'JAK inhibitor', 'protein synthesis inhibitor', 'HDAC inhibitor', 
        'topoisomerase inhibitor', 'PARP inhibitor', 'ATPase inhibitor', 'retinoid receptor agonist', 'HSP inhibitor']

In [ ]:
# read the data 
import pandas as pd
import numpy as np
all_data = pd.read_csv('../data_for_image_based_model/fl_data.csv')

In [ ]:
# Drop the lines with DMSO 
all_data = all_data[all_data.moa != 'dmso']
assert 'dmso' not in all_data.values

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [ ]:
dictionary = {'ATPase inhibitor': 7, 'Aurora kinase inhibitor': 0,
 'HDAC inhibitor': 4, 'HSP inhibitor': 9, 'JAK inhibitor': 2, 'PARP inhibitor': 6,
 'protein synthesis inhibitor': 3, 'retinoid receptor agonist': 8,
 'topoisomerase inhibitor': 5, 'tubulin polymerization inhibitor': 1}

In [ ]:
# change moa to classes 
all_data['classes'] = None
for i in range(all_data.shape[0]):
    all_data.iloc[i, 11] = dictionary[all_data.iloc[i, 10]]

In [ ]:
# add 'digit' column as index 
all_data['digit'] = [i for i in range(0, all_data.shape[0])]

In [ ]:
# shuffle the data 
all_data = all_data.sample(frac = 1, random_state = 1)

In [ ]:
# get the compounds in test data  
compound_test =  ['CBK303888', 'CBK200949', 'CBK309016', 'CBK290799', 'CBK308876', 'CBK289882',
 'CBK308108', 'CBK290852', 'CBK307956', 'CBK290529', 'CBK309488', 'CBK290717', 'CBK277957', 'CBK290547',
 'CBK291084', 'CBK289740', 'CBK278064', 'CBK308819', 'CBK308608', 'CBK288281', 'CBK309437', 'CBK289987',
 'CBK278556G', 'CBK277961']
assert len(compound_test) == 24 

In [ ]:
# get the compounds in validation data 
compound_valid = ['CBK309483', 'CBK200593', 'CBK278120', 'CBK309545', 'CBK277955', 'CBK278047', 'CBK278016', 'CBK290481', 'CBK290217',
 'CBK041182', 'CBK271643', 'CBK041804', 'CBK042154', 'CBK308980', 'CBK290803', 'CBK309258', 'CBK290154',
 'CBK308260', 'CBK017131', 'CBK290770', 'CBK291055', 'CBK307747', 'CBK228145']
assert len(compound_valid) == 23

In [ ]:
# get the compounds in training data 
compound_train = []
for i in all_data.compound.tolist():
    if i not in (compound_test + compound_valid + compound_train):
        compound_train.append(i)

assert len(compound_train) + len(compound_valid) + len(compound_test) == len(set(all_data.compound.to_list()))

In [ ]:
# get the train, valid and test set 
train = all_data[all_data['compound'].isin(compound_train)].reset_index(drop = True)  
valid = all_data[all_data['compound'].isin(compound_valid)].reset_index(drop = True)  
test = all_data[all_data['compound'].isin(compound_test)].reset_index(drop = True)  

In [ ]:
# get the dictionary for compound_id-SMILES pair 
import pickle
compound_smiles_dictionary = pickle.load(open("../data_for_image_based_model/dictionary2.pickle", "rb"))

In [ ]:
# on the fly data augmentation 
import albumentations as A
train_transforms = A.Compose([A.Flip(),A.ShiftScaleRotate(scale_limit=0.2),A.RandomRotate90(),
    A.OneOf([A.Flip(),A.ShiftScaleRotate(scale_limit=0.2),A.RandomRotate90(),],p = 0.2),
    A.OneOf([A.Flip(),A.ShiftScaleRotate(scale_limit=0.2),A.RandomRotate90(),],p = 0.4),
    A.OneOf([A.Flip(),A.ShiftScaleRotate(scale_limit=0.2),A.RandomRotate90(),],p = 0.5),
    A.OneOf([A.Flip(),A.ShiftScaleRotate(scale_limit=0.2),A.RandomRotate90(),],p = 0.6),
    A.OneOf([A.Flip(),A.ShiftScaleRotate(scale_limit=0.2),A.RandomRotate90(),],p = 0.8),
    A.Flip(),A.ShiftScaleRotate(scale_limit=0.2),A.RandomRotate90(),])
valid_transforms = A.Compose([])

In [ ]:
# get all images  
import numpy as np
all_images = np.load(open(, "rb")) # The path has to be set

In [ ]:
# data generator for training 
def get_train_image(end):
  start = 0
  while start < end:      
    idx = start       
    row = train.iloc[idx]

    assert row['digit']     in train.digit.tolist()
    assert row['compound']   in train.compound.tolist()

    assert row['digit']    not in valid.digit.tolist()
    assert row['compound']  not in valid.compound.tolist()

    assert row['digit']    not in test.digit.tolist()
    assert row['compound']  not in test.compound.tolist()

    image = all_images[all_data.digit.tolist().index(row['digit'])]  
    image = train_transforms(image = image)['image']               
    target = int(row['classes'])   
    
    yield image, target
    start += 1

In [ ]:
# data generator for validation   
def get_valid_image(end):
  start = 0
  while start<end:
    idx = start       
    row = valid.iloc[idx]

    assert row['digit']   not in train.digit.tolist()
    assert row['compound']  not in train.compound.tolist()

    assert row['digit']     in valid.digit.tolist()
    assert row['compound']   in valid.compound.tolist()

    assert row['digit']    not in test.digit.tolist()
    assert row['compound']  not in test.compound.tolist()
       
    image = all_images[all_data.digit.tolist().index(row['digit'])] 
    image = valid_transforms(image = image)['image']
    target = int(row['classes'])   
    
    yield image, target
    start += 1

In [ ]:
batch_size = 32
channels = 5      
image_size = 256 

In [ ]:
# turn to tensorflow datasets 
import tensorflow as tf
train_data = tf.data.Dataset.from_generator(get_train_image,
            (tf.float32, tf.int32),
            ((tf.TensorShape([image_size, image_size, channels])), tf.TensorShape([])),
            args = [train.shape[0]]).batch(batch_size, num_parallel_calls = 64).prefetch(1024)

valid_data = tf.data.Dataset.from_generator(get_valid_image,
            (tf.float32, tf.int32),
            ((tf.TensorShape([image_size, image_size, channels])), tf.TensorShape([])),
            args = [valid.shape[0]]).batch(batch_size, num_parallel_calls = 64).prefetch(1024)

In [ ]:
# we choose efficientnet b1 as the base model    
base_model = tf.keras.applications.EfficientNetB1(input_shape = (image_size, image_size, channels),
                                  include_top = False, weights = None)

In [ ]:
import tensorflow_addons as tfa
from tensorflow.keras import layers, models, optimizers, regularizers
from keras.layers import Dropout

In [ ]:
# complete the architecture of efficientnet b1
drop = 0.30
num_classes = len(set(train['classes'].tolist())) 
x = base_model.output
x = Dropout(drop)(x)
x = layers.GlobalAveragePooling2D()(x)
x = Dropout(drop)(x)
preds = layers.Dense(num_classes, activation = 'softmax',                     
    kernel_regularizer = regularizers.L1L2(l1 = 1e-4, l2 = 1e-3),
    bias_regularizer = regularizers.L2(1e-3),
    activity_regularizer = regularizers.L2(1e-4))(x)
cnn_model = models.Model(inputs = base_model.input, outputs = preds)

In [ ]:
# set the optimizer of efficientnet b1    
cnn_optimizer = tfa.optimizers.AdamW(weight_decay = 1e-6, learning_rate = 0.001, beta_1 = 0.9,
    beta_2 = 0.999, epsilon = 1e-07,)

In [ ]:
# compile the model   
cnn_model.compile(optimizer = cnn_optimizer,
         loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
         metrics = ['accuracy'])

In [ ]:
# set the class weights 
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced', 
         classes = np.unique(train.classes), y = train.classes)   
weight_dictionary = dict(zip(np.unique(train.classes), class_weights))

In [ ]:
# set the check point   
from keras.callbacks import ModelCheckpoint
filepath_cnn = './content/shuffle_5_' + str(base_model.name) + '_weights.hdf5'
checkpoint_cnn = ModelCheckpoint(filepath_cnn, monitor = 'val_accuracy', verbose = 0, 
                  save_best_only = True, mode = 'max')

In [ ]:
# train the efficientnet b1
from tensorflow.keras.callbacks import EarlyStopping  
reduce_lr_loss_cnn = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss',
              factor = 0.5, patience = 9, verbose = 2, min_lr = 1e-7, mode = 'min')
history_cnn = cnn_model.fit(train_data, validation_data = valid_data, class_weight = weight_dictionary,
               verbose = 2, epochs = 100, callbacks=[reduce_lr_loss_cnn, checkpoint_cnn,])

In [ ]:
# get the best model 
from keras.models import load_model
from sklearn.metrics import classification_report
best_cnn_model = load_model() # Path has to be set

In [ ]:
# evaluate the model   
predicted_test = []
for i in range(test.shape[0]):    
  row = test.iloc[i]

  assert row['digit']  not  in train.digit.tolist()
  assert row['compound'] not  in train.compound.tolist()
  
  assert row['digit']   not  in valid.digit.tolist()
  assert row['compound']  not  in valid.compound.tolist()

  assert row['digit']     in test.digit.tolist()
  assert row['compound']    in test.compound.tolist()  

  im = all_images[all_data.digit.tolist().index(row['digit'])] 
  im = valid_transforms(image = im)['image']
  im = np.expand_dims(im, 0)
  value = best_cnn_model.predict(im).argmax()
  
  predicted_test.append(value)   
  
print(classification_report(test.classes.tolist(), predicted_test))    

In [ ]:
# references  
# https://www.tensorflow.org/guide/data
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
# https://www.tensorflow.org/